In [1]:
#install nfl_data_py package
#!pip install nfl_data_py


In [2]:
import nfl_data_py as nfl


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from tqdm import tqdm

import nfl_data_py as nfl

%matplotlib inline

In [4]:
from sklearn.metrics import mean_absolute_error

In [5]:
#Calculate the market-implied team total, derived the betting lines

def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [6]:
def last_name(full):
  if full == 'DanielThomas':
    return 'Thomas'
  elif full == 'JulioJones':
    return 'Jones'
  elif '.' not in full:
    print(full)
    return full
  else:
    return full.split(".",1)[1]

In [7]:
def team_fixer(team):
    if team == 'SD':
        return 'LAC'
    elif team == 'OAK':
        return 'LV'
    elif team == 'STL':
        return 'LA'
    else:
        return team

In [8]:
def snap_name_changer(player):
    if 'Jr.' in player:
        return player.split(' Jr.')[0]
        #return 'D.J. Davis'
    elif 'II' in player:
        return player.split(' II')[0]
    else:
        return player

In [9]:
def position_handler(pos):
  if pos in ['WR','RB','TE','FB']:
    return pos
  elif pos == 'RB/W':
    return 'RB'
  elif pos == 'WR/R':
    return 'WR'
  elif pos == 'HB':
    return 'RB'
  else:
    return 'other'

In [10]:
import joblib
import pickle


play_by_play_model = pickle.load(open('play_by_play.pkl', 'rb'))


In [11]:
in_season_model = pickle.load(open('beta_model.pkl', 'rb'))

In [12]:
linear_model = pickle.load(open('linear_model.pkl', 'rb'))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
from tqdm import tqdm
import tqdm.notebook as tq
import warnings
warnings.filterwarnings('ignore')

# Apply Model

In [14]:
#load in data for NFL pass attempts

szn_receivers=[]
target_list=[]
  
df_list = []
weekly_list = []
target_week = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
week_list1= []


all_throws = []
n_szn = []
plus_1_szn = []

for YEAR in tqdm(range(2022,2023)):
    print(YEAR)
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    #data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    throws = data[data['air_yards'].isna()==False]
    df = throws[['receiver_player_name','week','posteam','game_id','complete_pass','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','pass_touchdown','down','pass_location','season','home_implied_total','away_implied_total','posteam_type','cp','xyac_mean_yardage','qb_hit','yards_after_catch']]

    current_szn_df = df.dropna(subset=['receiver_player_name'])

  #Create column for throws to the middle (1 for throws to middle, 0 for throws elsewhere)                               
    current_szn_df['middle'] = (current_szn_df['pass_location']=='middle')*1 
    current_szn_df['goal_to_go'] = (current_szn_df['yardline_100']<=10)*1 

    current_szn_df = current_szn_df[['receiver_player_name','week','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit','complete_pass','yards_after_catch','cp','xyac_mean_yardage','goal_to_go']].set_index(['receiver_player_name','posteam'])
    current_szn_df['year'] = YEAR
  #apply model to the season's stats
    current_x = current_szn_df.drop(['yards_gained','year','week','goal_to_go','complete_pass','yards_after_catch','cp','xyac_mean_yardage'],axis=1)
    current_szn_df['xYards']=play_by_play_model.predict(current_x)
    current_szn_df = current_szn_df.reset_index()
  
    
    yac_df = current_szn_df[current_szn_df['complete_pass']==1]
    weekly_yac = yac_df.groupby(['receiver_player_name','posteam','week']).agg({'complete_pass':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum'})
    
    
    
    weekly_pass_offense=current_szn_df.groupby(['posteam','week']).agg({'week':'count','air_yards':'sum'})
    weekly_pass_offense.rename(columns={'week':'team_attempts','air_yards':'team_air_yards'},inplace=True)
    team_attempts=weekly_pass_offense.reset_index()[['posteam','week','team_attempts','team_air_yards']]
    team_attempts.set_index(['posteam','week'],inplace=True)
    weekly_rec_group = current_szn_df.groupby(['receiver_player_name','posteam','week']).agg({'week':'max','down':'count','complete_pass':'sum','yards_gained':'sum','air_yards':'sum','xYards':'sum','cp':'sum','implied_posteam_total':'max','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum'})
    weekly_rec_group.rename(columns={'week':'week#','down':'targets'},inplace=True)
    weekly_rec_group = weekly_rec_group.merge(weekly_yac[['yards_after_catch','xyac_mean_yardage']],right_index=True,left_index=True)
    
    
    feature_receivers = weekly_rec_group.reset_index().set_index(['posteam','week']).merge(team_attempts,right_index=True,left_index=True)
    feature_receivers = feature_receivers.reset_index().set_index(['receiver_player_name','posteam','week'])
    feature_receivers['szn'] = YEAR
    
    week_df1 = feature_receivers.copy()
    week_df = week_df1.copy()
    week_df = week_df.reset_index().set_index(['receiver_player_name','posteam'])
    feature_df = feature_receivers.copy()
    
    for target in tqdm(target_week):
        feature_receivers = feature_df[feature_df['week#']<target]
    
        player_list = []

        for x in feature_receivers.reset_index().set_index(['receiver_player_name','posteam']).index:
            player_list.append(x)
    
    
        players = list(set(player_list))

        print(target)
        for player in players:
            feature_receivers['xYards_median'] = feature_receivers['xYards']
            weekly_player = pd.DataFrame(feature_receivers.loc[player].sort_values('week#',ascending=False).head(6).agg({'yards_gained':'sum','targets':'sum','complete_pass':'sum','week#':'count','szn':'max','air_yards':'sum','xYards':'sum','cp':'sum','goal_to_go':'sum','qb_hit':'sum','middle':'sum','goal_to_go':'sum','team_attempts':'sum','team_air_yards':'sum','yards_after_catch':'sum','xyac_mean_yardage':'sum','xYards_median':'median'})).transpose()
            weekly_player['player'] = player[0]
            weekly_player['team'] = player[1]
            weekly_player['target_week'] = target
            weekly_player.set_index(['player','team','target_week','szn'],inplace=True)
            
          
            df_list.append(weekly_player)
        
        week_df = week_df1.copy()
        
        week_df = week_df[week_df['week#']==target]
        week_df = week_df.reset_index().rename(columns={'week':'target_week','receiver_player_name':'player','posteam':'team'})
        week_df = week_df.set_index(['player','team','target_week','szn'])

        week_list1.append(week_df)
        
        
        
        
        
weekly_receivers = pd.concat(df_list)
weekly_targets = pd.concat(week_list1)
        
        #weekly_receivers['aDOT']=weekly_receivers['air_yards']/weekly_receivers['targets']
        #weekly_receivers['AY_share']=round(weekly_receivers['air_yards']/weekly_receivers['team_air_yards'],3)
        #weekly_receivers['target_share']=round(weekly_receivers['targets']/weekly_receivers['team_attempts'],3)
        #weekly_receivers['yac/rec']=round(weekly_receivers['yards_after_catch']/weekly_receivers['complete_pass'],3)
        #weekly_receivers['xyac/rec']=round(weekly_receivers['xyac_mean_yardage']/weekly_receivers['complete_pass'],3)


        
        

  0%|                                                     | 0/1 [00:00<?, ?it/s]

2022



  0%|                                                    | 0/18 [00:00<?, ?it/s]

1
2



 11%|████▉                                       | 2/18 [00:04<00:34,  2.19s/it]

...


 89%|██████████████████████████████████████▏    | 16/18 [01:48<00:16,  8.09s/it]

17



 94%|████████████████████████████████████████▌  | 17/18 [01:57<00:08,  8.18s/it]

18



100%|████████████████████████████████████████████| 1/1 [02:42<00:00, 162.80s/it]


In [15]:
feature_df['target_week'] = feature_df['week#']+1

feature_df.drop(('A.St. Brown','DET',7),inplace=True)

In [16]:
feature_df.loc['T.Lockett']

week#  targets  complete_pass  yards_gained  air_yards  \
posteam week                                                           
SEA     1         1        4            3.0          28.0       29.0   
        2         2       11            9.0         107.0      118.0   
        3         3       11            9.0          76.0      123.0   
        4         4        8            6.0          91.0      122.0   
        5         5        6            5.0         104.0      135.0   
        6         6        5            2.0          17.0       62.0   
        7         7        8            7.0          45.0       43.0   
        8         8        8            5.0          63.0       84.0   
        9         9        5            5.0          67.0       24.0   
        10       10        5            3.0          42.0       53.0   
        12       12        7            3.0          68.0       90.0   
        13       13       12            9.0         128.0       89.0   
        14       14        9            5.0          60.0       97.0   
        15       15        9            7.0          68.0       67.0   
        17       17        2            2.0          15.0        8.0   
        18       18        7            4.0          54.0       96.0   

                 xYards        cp  implied_posteam_total  goal_to_go  qb_hit  \
posteam week                                                                   
SEA     1     29.970425  2.799239                  19.00           0     0.0   
        2     80.492153  6.903809                  15.75           0     2.0   
        3     84.512757  6.664902                  22.00           0     1.0   
        4     65.110391  4.626942                  22.75           1     2.0   
        5     51.207771  3.103960                  20.00           0     2.0   
        6     39.778139  3.077915                  23.75           0     0.0   
        7     52.408489  5.625344                  23.00           0     1.0   
        8     51.552411  4.539161                  23.75           0     2.0   
        9     31.774134  3.406899                  24.00           1     1.0   
        10    37.522324  3.040777                  21.25           0     0.0   
        12    64.875953  4.536230                  25.75           0     1.0   
        13    89.451189  8.285791                  24.00           1     1.0   
        14    72.222910  5.002261                  24.00           1     1.0   
        15    66.285869  6.362094                  20.00           0     0.0   
        17    12.863331  1.584767                  21.00           0     0.0   
        18    61.932555  4.079475                  23.50           0     1.0   

              middle  yards_after_catch  xyac_mean_yardage  team_attempts  \
posteam week                                                                
SEA     1          1                5.0          15.557277             27   
        2          3               22.0          36.641333             30   
        3          0                3.0          34.271528             44   
        4          2               11.0          22.844684             29   
        5          2                5.0          10.438336             24   
        6          2                8.0           7.939587             30   
        7          1                9.0          34.396273             26   
        8          0               28.0          27.092477             33   
        9          1               43.0          27.223590             32   
        10         1                9.0           8.453819             33   
        12         5               24.0           9.173747             37   
        13         2               68.0          43.275493             37   
        14         0                1.0          18.435113             36   
        15         1               21.0          31.932819             41   
        17         1                7.0     

In [17]:
    YEAR=2022
    
    
    snap_df = nfl.import_snap_counts([YEAR])
    snap_df['team'] = snap_df['team'].apply(lambda x: team_fixer(x))
    snap_df = snap_df[snap_df['game_type']=='REG']
    snap_df = snap_df[['pfr_player_id','player','offense_snaps','offense_pct','week','team','game_id','position']]
    snap_df['player_copy'] = snap_df['player']
    snap_df.rename(columns={'player_copy':'full_name'},inplace=True)
    snap_df['full_name'] = snap_df['full_name'].apply(lambda x: snap_name_changer(x))
    
    
    snap_df['week#'] = snap_df['week']
    #snap_df = snap_df.set_index(['receiver_player_id','team','week'])
    snap_df = snap_df[snap_df['offense_snaps']>0]

    snap_df['last_name'] = snap_df['full_name'].apply(lambda x: x.split(" ",1)[1])
    snap_df['identifier'] = (snap_df['full_name']).str[0]+"."+snap_df['last_name']+"_"+snap_df['team']+"_"+(snap_df['week'].astype(str))
    snap_df = snap_df[['full_name','identifier','offense_snaps','position','team']]




In [18]:
snap_df = snap_df[snap_df['identifier']!='A.St. Brown_DET_7']

snap_df = snap_df[snap_df['identifier']!='D.Metcalf_SEA_7']

In [19]:
snap_df = snap_df[snap_df['identifier']!='M.Andrews_BAL_8']


In [20]:
player_df_list = []



for player_id in tq.tqdm(feature_df.index):

  player = player_id[0:2]
  player_df = feature_df.loc[player]
  week_df=pd.DataFrame()
  week_df['week#']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
  player_df = player_df.merge(week_df,how='right')
  player_df['receiver_player_name'] = player[0]
  
  player_df['posteam'] = player[1]
  player_df['last'] = player_df['receiver_player_name'].apply(lambda x: last_name(x))
  player_df['identifier'] = (player_df['receiver_player_name']).str[0]+"."+player_df['last']+"_"+player_df['posteam']+"_"+player_df['week#'].astype(str)
  player_df = player_df.merge(snap_df,on='identifier')



  player_df.fillna(0,inplace=True)

  player_df['xYards_median'] = player_df['xYards'].rolling(6,min_periods=6).median()
  player_df['xYards/game'] = player_df['xYards'].rolling(6,min_periods=6).mean()

  player_df['xYards_sum'] = player_df['xYards'].rolling(6,min_periods=6).sum()
  player_df['yards_sum'] = player_df['yards_gained'].rolling(6,min_periods=6).sum()
  player_df['yards/game'] = player_df['yards_gained'].rolling(6,min_periods=6).mean()
  player_df['games'] = player_df['week#'].rolling(6,min_periods=1).count()
  player_df.rename(columns={'week#':'week'},inplace=True)
  target_df1 = pd.DataFrame()


  player_df['target_week'] = player_df['week']+1

  target_df1['target_week']=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

  player_df = player_df.merge(target_df1,how='right').sort_values('target_week')




  player_df = player_df.fillna(method='ffill')

  player_df_list.append(player_df)




  0%|          | 0/3860 [00:00<?, ?it/s]

In [21]:
trailing_weeks = pd.concat(player_df_list)


In [22]:
new_df = trailing_weeks[['receiver_player_name','full_name','xYards','yards_gained','week','posteam']].drop_duplicates()



In [23]:
new_df = new_df[new_df['receiver_player_name'].isna()==False]

## 5. Actual Results

### Week 8

In [24]:
week=8

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6


trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])

#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])


trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,E.Benjamin,16.568489,17.815921,5.466466,22.693298,18.000989,23.951324,ARI,17.416081,20.0,7.0,9.0,28.0,28.0,21.0,18.833333,21.725853
1,G.Dortch,25.054535,63.685232,6.667368,0.000000,0.000000,2.921464,ARI,16.388100,55.0,80.0,6.0,0.0,0.0,5.0,24.333333,15.683463
2,M.Brown,54.376546,80.610047,124.714429,84.334507,92.688104,102.644805,ARI,89.894740,43.0,68.0,140.0,88.0,78.0,68.0,80.833333,75.225423
3,Z.Ertz,66.009906,63.140109,38.573004,82.295881,88.229569,31.020628,ARI,61.544850,75.0,45.0,47.0,48.0,70.0,21.0,51.000000,53.771736
4,S.Anderson,0.000000,5.805800,0.000000,0.000000,0.000000,0.000000,ARI,0.967633,0.0,2.0,0.0,0.0,0.0,0.0,0.333333,4.548942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,T.McLaurin,77.558318,80.633753,43.355203,53.357518,23.122408,57.285837,WAS,55.885506,75.0,102.0,15.0,76.0,41.0,73.0,63.666667,54.813953
253,D.Milne,4.955082,7.600840,0.000000,0.000000,6.859129,3.224259,WAS,3.773218,6.0,12.0,0.0,0.0,5.0,4.0,4.500000,6.003890
254,J.Bates,0.000000,21.277663,0.000000,29.472004,23.062004,0.000000,WAS,12.301945,0.0,15.0,0.0,19.0,39.0,0.0,12.166667,13.239272
255,D.Brown,0.000000,0.000000,0.000000,32.931990,49.563682,0.000000,WAS,13.749279,0.0,0.0,0.0,6.0,105.0,0.0,18.500000,20.842170


In [25]:
yardage_trailing.reset_index().rename(columns={'index':'player'})


,player,player,1a,2a,3a,4a,5a,6a,team
0,0,E.Benjamin,20.0,7.0,9.0,28.0,28.0,21.0,ARI
1,1,G.Dortch,55.0,80.0,6.0,0.0,0.0,5.0,ARI
2,2,M.Brown,43.0,68.0,140.0,88.0,78.0,68.0,ARI
3,3,Z.Ertz,75.0,45.0,47.0,48.0,70.0,21.0,ARI
4,4,S.Anderson,0.0,2.0,0.0,0.0,0.0,0.0,ARI
...,...,...,...,...,...,...,...,...,...
252,252,T.McLaurin,75.0,102.0,15.0,76.0,41.0,73.0,WAS
253,253,D.Milne,6.0,12.0,0.0,0.0,5.0,4.0,WAS
254,254,J.Bates,0.0,15.0,0.0,19.0,39.0,0.0,WAS
255,255,D.Brown,0.0,0.0,0.0,6.0,105.0,0.0,WAS


In [26]:
projections = trailing_weeks.sort_values('pYards',ascending=False)

#[['player','team','pYards','Yards/game','games','target_share_x','aDOT','xYards/game','team_attempts_x']]

In [27]:
df=pd.read_csv('betting_lines/nfl-best-bets.csv')
df = df[df['team']!='CLV']

mnf=pd.read_csv('betting_lines/nfl-best-bets (1).csv')

In [28]:
df = df.append(mnf)

In [29]:
projections.reset_index(inplace=True)

#projections.rename(columns={'index':'player'},inplace=True)

In [30]:
names_to_change = ['M.Jones Jr','E.St','I.Smith Jr','A.St','T.Marshall Jr','A.Robinson II','M.Pittman Jr','K.Walker III','M.Gordon III']

def name_changer(name):
    if name in names_to_change:
        if name == 'M.Jones Jr':
            return 'M.Jones'
        if name == 'E.St':
            return 'E.St. Brown'
        if name == 'I.Smith Jr':
            return 'I.Smith'
        if name == 'A.St':
            return 'A.St. Brown'
        if name == 'T.Marshall Jr':
            return 'T.Marshall'
        if name == 'A.Robinson II':
            return 'Al.Robinson'
        if name == 'M.Pittman Jr':
            return 'M.Pittman'
        if name == 'K.Walker III':
            return 'K.Walker'
        if name == 'M.Gordon III':
            return 'M.Gordon'
    if name not in names_to_change:
        return name

In [31]:
odds = df[['player','team','line']]

def team_changer(name):
    if name == 'HST':
        return 'HOU'
    elif name == 'CLV':
        return 'CLE'
    elif name == 'ARZ':
        return 'ARI'
    elif name == 'BLT':
        return 'BAL'
    else:
        return name

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))



odds


,player,team,line
0,C.Sutton,DEN,54.5
1,G.Dulcich,DEN,33.5
2,J.Jeudy,DEN,47.5
3,K.Hamler,DEN,17.5
4,L.Murray,DEN,6.5
...,...,...,...
6,A.Cooper,CLE,53.5
7,D.Bell,CLE,15.5
8,D.Peoples-Jones,CLE,44.5
9,K.Hunt,CLE,12.5


In [32]:
odds

,player,team,line
0,C.Sutton,DEN,54.5
1,G.Dulcich,DEN,33.5
2,J.Jeudy,DEN,47.5
3,K.Hamler,DEN,17.5
4,L.Murray,DEN,6.5
...,...,...,...
6,A.Cooper,CLE,53.5
7,D.Bell,CLE,15.5
8,D.Peoples-Jones,CLE,44.5
9,K.Hunt,CLE,12.5


In [33]:
projections

,index,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,161,T.Hill,106.486919,39.371270,118.805974,44.580312,128.744770,138.465731,MIA,96.075829,190.0,33.0,160.0,47.0,177.0,72.0,113.166667,93.205954
1,24,S.Diggs,65.873396,108.425721,93.087057,62.928860,84.346956,106.733110,BUF,86.899183,122.0,148.0,74.0,62.0,102.0,148.0,109.333333,87.251667
2,168,J.Jefferson,82.318891,75.323501,42.299787,94.376820,103.337013,62.127839,MIN,76.630642,184.0,48.0,14.0,136.0,154.0,107.0,107.166667,85.129546
3,138,C.Kupp,111.365605,98.016954,59.620847,114.653996,76.448321,60.196098,LA,86.716970,128.0,108.0,44.0,122.0,125.0,80.0,101.166667,84.899368
4,235,M.Evans,62.504047,42.754883,81.694037,78.004820,34.698985,153.712147,TB,75.561487,71.0,61.0,113.0,81.0,42.0,96.0,77.333333,83.634565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,147,T.McKitty,0.000000,0.000000,6.441841,0.000000,0.000000,0.000000,LAC,1.073640,0.0,0.0,4.0,0.0,0.0,0.0,0.666667,4.548942
253,218,D.Watt,0.000000,0.000000,0.000000,4.874831,0.000000,0.000000,PIT,0.812472,0.0,0.0,0.0,1.0,0.0,0.0,0.166667,4.548942
254,132,M.Burton,6.077989,0.000000,0.000000,0.000000,0.000000,0.000000,KC,1.012998,2.0,0.0,0.0,0.0,0.0,0.0,0.333333,4.548942
255,78,R.Wilson,0.000000,0.000000,0.000000,5.308920,0.000000,0.000000,DEN,0.884820,0.0,0.0,0.0,1.0,0.0,0.0,0.166667,4.548942


In [34]:
projections

,index,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,161,T.Hill,106.486919,39.371270,118.805974,44.580312,128.744770,138.465731,MIA,96.075829,190.0,33.0,160.0,47.0,177.0,72.0,113.166667,93.205954
1,24,S.Diggs,65.873396,108.425721,93.087057,62.928860,84.346956,106.733110,BUF,86.899183,122.0,148.0,74.0,62.0,102.0,148.0,109.333333,87.251667
2,168,J.Jefferson,82.318891,75.323501,42.299787,94.376820,103.337013,62.127839,MIN,76.630642,184.0,48.0,14.0,136.0,154.0,107.0,107.166667,85.129546
3,138,C.Kupp,111.365605,98.016954,59.620847,114.653996,76.448321,60.196098,LA,86.716970,128.0,108.0,44.0,122.0,125.0,80.0,101.166667,84.899368
4,235,M.Evans,62.504047,42.754883,81.694037,78.004820,34.698985,153.712147,TB,75.561487,71.0,61.0,113.0,81.0,42.0,96.0,77.333333,83.634565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,147,T.McKitty,0.000000,0.000000,6.441841,0.000000,0.000000,0.000000,LAC,1.073640,0.0,0.0,4.0,0.0,0.0,0.0,0.666667,4.548942
253,218,D.Watt,0.000000,0.000000,0.000000,4.874831,0.000000,0.000000,PIT,0.812472,0.0,0.0,0.0,1.0,0.0,0.0,0.166667,4.548942
254,132,M.Burton,6.077989,0.000000,0.000000,0.000000,0.000000,0.000000,KC,1.012998,2.0,0.0,0.0,0.0,0.0,0.0,0.333333,4.548942
255,78,R.Wilson,0.000000,0.000000,0.000000,5.308920,0.000000,0.000000,DEN,0.884820,0.0,0.0,0.0,1.0,0.0,0.0,0.166667,4.548942


In [35]:
table = odds.merge(projections,on=['player','team'])

table = table.round(2)

table['diff'] = table['pYards'] - table['line']




table['abs_diff'] = abs(table['diff'])

In [36]:
table.sort_values('pYards',ascending=False).head(50)

,player,team,line,index,1,2,3,4,5,6,...,1a,2a,3a,4a,5a,6a,yards/game,pYards,diff,abs_diff
29,T.Hill,MIA,88.5,161,106.49,39.37,118.81,44.58,128.74,138.47,...,190.0,33.0,160.0,47.0,177.0,72.0,113.17,93.21,4.71,4.71
86,S.Diggs,BUF,80.5,24,65.87,108.43,93.09,62.93,84.35,106.73,...,122.0,148.0,74.0,62.0,102.0,148.0,109.33,87.25,6.75,6.75
20,J.Jefferson,MIN,89.5,168,82.32,75.32,42.30,94.38,103.34,62.13,...,184.0,48.0,14.0,136.0,154.0,107.0,107.17,85.13,-4.37,4.37
65,C.Kupp,LA,92.5,138,111.37,98.02,59.62,114.65,76.45,60.20,...,128.0,108.0,44.0,122.0,125.0,80.0,101.17,84.90,-7.60,7.60
48,A.Brown,PHI,69.5,203,100.78,82.17,78.39,72.92,52.57,54.53,...,155.0,69.0,85.0,95.0,32.0,67.0,83.83,77.96,8.46,8.46
46,C.Olave,NO,66.5,180,28.74,136.88,102.31,65.40,52.62,124.85,...,41.0,80.0,147.0,67.0,54.0,106.0,82.50,75.67,9.17,9.17
44,D.Adams,LV,83.5,151,126.43,50.86,65.54,94.19,71.94,76.94,...,141.0,12.0,36.0,101.0,124.0,95.0,84.83,73.68,-9.82,9.82
14,C.Lamb,DAL,71.5,63,79.80,88.36,66.06,55.97,61.77,68.70,...,75.0,87.0,97.0,53.0,68.0,70.0,75.00,72.11,0.61,0.61
71,M.Pittman,IND,57.5,106,112.92,63.14,30.15,62.82,109.13,63.38,...,121.0,72.0,31.0,59.0,134.0,58.0,79.17,66.44,8.94,8.94
25,J.Waddle,MIA,64.5,158,144.01,69.49,45.14,28.38,101.90,53.21,...,171.0,102.0,39.0,23.0,129.0,88.0,92.00,65.90,1.40,1.40


In [37]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]

  

100%|█████████████████████████████████████████████| 1/1 [00:06<00:00,  6.56s/it]


In [38]:
#table.shape[0]


table = table.drop_duplicates()

In [39]:
table['pYards'].corr(table['line'])


0.9529384455175356

In [40]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==8]



100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.23s/it]


In [41]:
weekly_Rec = results.groupby('receiver_player_name').agg({'yards_gained':'sum','pass':'count','complete_pass':'sum'}).reset_index().rename(columns={'receiver_player_name':'player'})

In [42]:
weekly_Rec

,player,yards_gained,pass,complete_pass
0,A.Abdullah,28.0,5,4.0
1,A.Brown,156.0,10,6.0
2,A.Cooper,131.0,7,5.0
3,A.Dillon,9.0,1,1.0
4,A.Gibson,58.0,7,7.0
...,...,...,...,...
236,W.Robinson,15.0,3,2.0
237,Z.Ertz,34.0,5,4.0
238,Z.Gentry,3.0,1,1.0
239,Z.Jones,28.0,3,3.0


In [43]:
table

,player,team,line,index,1,2,3,4,5,6,...,1a,2a,3a,4a,5a,6a,yards/game,pYards,diff,abs_diff
0,C.Sutton,DEN,54.5,71,96.22,81.00,60.07,78.82,19.46,66.03,...,122.0,97.0,52.0,74.0,14.0,23.0,63.67,60.45,5.95,5.95
1,J.Jeudy,DEN,47.5,72,29.79,56.20,36.56,61.24,65.08,75.00,...,11.0,17.0,53.0,53.0,54.0,96.0,47.33,49.99,2.49,2.49
2,K.Hamler,DEN,17.5,76,0.00,0.00,16.18,14.91,33.69,35.42,...,0.0,0.0,55.0,10.0,44.0,4.0,18.83,22.20,4.70,4.70
3,M.Gordon,DEN,12.5,74,8.09,32.61,0.00,16.81,0.00,21.89,...,6.0,29.0,0.0,49.0,0.0,17.0,16.83,15.95,3.45,3.45
4,C.Kirk,JAX,52.5,111,38.28,56.24,68.55,31.45,26.13,88.44,...,78.0,72.0,60.0,5.0,15.0,96.0,54.33,53.15,0.65,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,T.Boyd,CIN,61.5,50,16.26,41.54,43.05,29.67,38.23,83.26,...,17.0,105.0,47.0,32.0,66.0,155.0,70.33,59.39,-2.11,2.11
96,A.Cooper,CLE,53.5,54,78.73,90.90,28.35,88.26,89.82,35.33,...,101.0,101.0,9.0,76.0,44.0,74.0,67.50,61.09,7.59,7.59
97,D.Peoples-Jones,CLE,44.5,57,0.00,21.30,69.93,60.46,42.54,41.68,...,0.0,10.0,71.0,50.0,74.0,71.0,46.00,43.21,-1.29,1.29
98,K.Hunt,CLE,12.5,59,10.95,23.77,15.26,15.07,0.00,11.01,...,16.0,14.0,19.0,10.0,0.0,4.0,10.50,12.18,-0.32,0.32


In [44]:
#table.drop(columns={'yards_gained'},inplace=True)

In [45]:
smash_table = table[table['abs_diff']>=10]

results = table.merge(weekly_Rec,on='player')


In [46]:
results=results.dropna()



In [47]:
results['Over_Picked'] =results['pYards']>results['line']

In [48]:
results['Over_Result'] = results['yards_gained'] > results['line']

results['Correct'] = results['Over_Picked'] == results['Over_Result']

In [49]:
w8_results = results[['player','team','line','pYards','yards_gained','Over_Picked','Over_Result','Correct']]

w8_results['week'] = 8

w8_results = w8_results[['player','team','line','pYards','yards_gained','Over_Picked','Over_Result','Correct']]


In [50]:
results['yards_gained'].corr(results['pYards'])

0.6294879366071175

In [51]:
#0.5860590594191387

In [52]:
import math
MSE = np.square(np.subtract(results['yards_gained'],results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (players w/ listed odds):\n")
print(RMSE)

print('-------------')

weekly_all  = weekly_Rec.merge(projections)[['player','pYards','yards_gained']]

MSE = np.square(np.subtract(weekly_all['yards_gained'],weekly_all['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error (all players):\n")
print(RMSE)

Root Mean Square Error (players w/ listed odds):

30.70305921073598
-------------
Root Mean Square Error (all players):

30.00196639198275


In [53]:
#30.880608939504032
#-------------

#30.24493362852292

In [54]:
#overall
results['Correct'].value_counts()

True     52
False    46
Name: Correct, dtype: int64

In [55]:
smash_results = results[results['abs_diff']>=10]

smash_results['Correct'].value_counts()

False    7
True     6
Name: Correct, dtype: int64

In [56]:
smash_results[['player','line','xYards/game','pYards','abs_diff','yards_gained','Over_Picked','Over_Result','Correct']].sort_values('abs_diff',ascending=False)

,player,line,xYards/game,pYards,abs_diff,yards_gained,Over_Picked,Over_Result,Correct
6,M.Jones,28.5,55.08,51.22,22.72,0.0,True,False,False
92,T.Higgins,76.5,59.61,60.46,16.04,49.0,False,False,True
15,Z.Ertz,38.5,61.54,53.77,15.27,34.0,True,False,False
21,J.Reynolds,39.5,50.54,53.36,13.86,14.0,True,False,False
56,P.Freiermuth,35.5,50.46,48.79,13.29,57.0,True,True,True
58,B.Cooks,60.5,53.32,47.33,13.17,73.0,False,True,False
53,G.Pickens,40.5,48.59,52.85,12.35,0.0,True,False,False
46,M.Callaway,39.5,28.78,27.77,11.73,10.0,False,False,True
42,T.Conklin,19.5,31.30,31.04,11.54,79.0,True,True,True
79,M.Goodwin,35.5,18.11,23.96,11.54,33.0,False,False,True


In [57]:
picked_overs = results[results['Over_Picked']==True]

picked_overs['Correct'].value_counts()

True     34
False    30
Name: Correct, dtype: int64

In [58]:
picked_unders = results[results['Over_Picked']==False]

picked_unders['Correct'].value_counts()

True     18
False    16
Name: Correct, dtype: int64

In [59]:
results['Over_Result'].value_counts()

True     50
False    48
Name: Over_Result, dtype: int64

In [60]:
smash_overs = smash_results[smash_results['Over_Picked']==True]


smash_overs['Correct'].value_counts()

False    5
True     3
Name: Correct, dtype: int64

In [61]:
smash_unders = smash_results[smash_results['Over_Picked']==False]


smash_unders['Correct'].value_counts()

True     3
False    2
Name: Correct, dtype: int64

In [62]:
table[table['team']=='CLE']

,player,team,line,index,1,2,3,4,5,6,...,1a,2a,3a,4a,5a,6a,yards/game,pYards,diff,abs_diff
96,A.Cooper,CLE,53.5,54,78.73,90.90,28.35,88.26,89.82,35.33,...,101.0,101.0,9.0,76.0,44.0,74.0,67.50,61.09,7.59,7.59
97,D.Peoples-Jones,CLE,44.5,57,0.00,21.30,69.93,60.46,42.54,41.68,...,0.0,10.0,71.0,50.0,74.0,71.0,46.00,43.21,-1.29,1.29
98,K.Hunt,CLE,12.5,59,10.95,23.77,15.26,15.07,0.00,11.01,...,16.0,14.0,19.0,10.0,0.0,4.0,10.50,12.18,-0.32,0.32
99,N.Chubb,CLE,7.5,60,15.63,0.00,14.51,0.00,10.55,15.41,...,26.0,0.0,9.0,0.0,14.0,16.0,10.83,13.15,5.65,5.65


### Week 9

In [63]:
week=9

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,A.Green,35.595872,44.266282,0.000000,44.593091,23.525625,0.000000,ARI,24.663478,13.0,16.0,0.0,20.0,7.0,0.0,9.333333,19.049418
1,E.Benjamin,17.815921,5.466466,22.693298,18.000989,23.951324,40.473309,ARI,21.400218,7.0,9.0,28.0,28.0,21.0,23.0,19.333333,22.798088
2,G.Dortch,63.685232,6.667368,0.000000,0.000000,2.921464,19.001749,ARI,15.379302,80.0,6.0,0.0,0.0,5.0,12.0,17.166667,16.785567
3,M.Brown,54.376546,80.610047,124.714429,84.334507,92.688104,102.644805,ARI,89.894740,43.0,68.0,140.0,88.0,78.0,68.0,80.833333,75.225423
4,Z.Ertz,63.140109,38.573004,82.295881,88.229569,31.020628,38.834502,ARI,57.015616,45.0,47.0,48.0,70.0,21.0,34.0,44.166667,49.382358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,T.McLaurin,80.633753,43.355203,53.357518,23.122408,57.285837,61.407276,WAS,53.193666,102.0,15.0,76.0,41.0,73.0,113.0,70.000000,60.163920
287,D.Milne,7.600840,0.000000,0.000000,6.859129,3.224259,5.629935,WAS,3.885694,12.0,0.0,0.0,5.0,4.0,4.0,4.166667,5.533191
288,J.Bates,21.277663,0.000000,29.472004,23.062004,0.000000,5.433521,WAS,13.207532,15.0,0.0,19.0,39.0,0.0,6.0,13.166667,11.896218
289,D.Brown,0.000000,0.000000,0.000000,32.931990,49.563682,0.000000,WAS,13.749279,0.0,0.0,0.0,6.0,105.0,0.0,18.500000,20.842170


In [64]:
trailing_weeks.drop_duplicates().shape[0]

288

In [65]:
trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,A.Green,35.595872,44.266282,0.000000,44.593091,23.525625,0.000000,ARI,24.663478,13.0,16.0,0.0,20.0,7.0,0.0,9.333333,19.049418
1,E.Benjamin,17.815921,5.466466,22.693298,18.000989,23.951324,40.473309,ARI,21.400218,7.0,9.0,28.0,28.0,21.0,23.0,19.333333,22.798088
2,G.Dortch,63.685232,6.667368,0.000000,0.000000,2.921464,19.001749,ARI,15.379302,80.0,6.0,0.0,0.0,5.0,12.0,17.166667,16.785567
3,M.Brown,54.376546,80.610047,124.714429,84.334507,92.688104,102.644805,ARI,89.894740,43.0,68.0,140.0,88.0,78.0,68.0,80.833333,75.225423
4,Z.Ertz,63.140109,38.573004,82.295881,88.229569,31.020628,38.834502,ARI,57.015616,45.0,47.0,48.0,70.0,21.0,34.0,44.166667,49.382358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,T.McLaurin,80.633753,43.355203,53.357518,23.122408,57.285837,61.407276,WAS,53.193666,102.0,15.0,76.0,41.0,73.0,113.0,70.000000,60.163920
287,D.Milne,7.600840,0.000000,0.000000,6.859129,3.224259,5.629935,WAS,3.885694,12.0,0.0,0.0,5.0,4.0,4.0,4.166667,5.533191
288,J.Bates,21.277663,0.000000,29.472004,23.062004,0.000000,5.433521,WAS,13.207532,15.0,0.0,19.0,39.0,0.0,6.0,13.166667,11.896218
289,D.Brown,0.000000,0.000000,0.000000,32.931990,49.563682,0.000000,WAS,13.749279,0.0,0.0,0.0,6.0,105.0,0.0,18.500000,20.842170


In [66]:
w9_table = trailing_weeks.copy()


In [67]:
#w9_table = w9_table.reset_index().rename(columns={'index':'player'})

In [68]:
w9_table.set_index(['player','team'],inplace=True)

In [69]:
w9_odds = pd.read_csv('betting_lines/nfl-best-bets (3).csv').round(1)[['player','team','line']]
w9_odds = w9_odds[w9_odds['team']!='BLT']
w9_odds = w9_odds[w9_odds['team']!='NO']


w9_odds = w9_odds.append(pd.read_csv('betting_lines/nfl-best-bets (4).csv').round(1)[['player','team','line']])



odds = w9_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


In [70]:
odds.set_index(['player','team'],inplace=True)

In [71]:

w9_table = w9_table.merge(odds,right_index=True,left_index=True)
w9_table = w9_table.reset_index()

w9_table['abs. diff'] = abs(w9_table['pYards'] - w9_table['line'])

w9_table = w9_table.round(1)

w9_table['Over_Picked'] =w9_table['pYards']>w9_table ['line']

#w9_table.sort_values('abs. diff',ascending=False).head(50)


In [72]:
w9_table[w9_table['team']=='BAL']

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
19,D.Duvernay,BAL,12.5,27.7,62.3,33.6,24.7,23.1,30.7,25.0,51.0,54.0,14.0,42.0,31.0,36.2,31.9,42.5,10.6,False
34,I.Likely,BAL,10.4,0.0,8.8,25.6,9.5,55.8,18.3,8.0,0.0,7.0,30.0,16.0,77.0,23.0,23.5,41.5,18.0,False
43,J.Oliver,BAL,6.6,0.0,0.0,25.6,0.0,6.8,6.5,8.0,0.0,0.0,21.0,0.0,4.0,5.5,8.1,7.5,0.6,True
48,K.Drake,BAL,6.0,0.0,5.9,12.7,0.0,17.1,6.9,15.0,0.0,1.0,8.0,0.0,5.0,4.8,10.1,12.5,2.4,False


In [73]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==9]


100%|█████████████████████████████████████████████| 1/1 [00:06<00:00,  6.77s/it]


In [74]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w9_table = w9_table.merge(results)

w9_table['Over_Result'] = w9_table['yards_gained']>w9_table['line']

w9_table['Win'] = (w9_table['Over_Result'] == w9_table['Over_Picked'])*1



In [75]:

import math
MSE = np.square(np.subtract(w9_table['pYards'],w9_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w9_table['yards_gained'],w9_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

26.26588189503901
-----
Root Mean Square Error - Betting Line:

24.883579500056502


In [76]:
w9_table['week'] = 9

In [77]:
w8_results['Correct'] = w8_results['Correct']*1
w8_results.rename(columns={'Correct':'Win'},inplace=True)

In [78]:
w8_results[w8_results['team']=='DET']

,player,team,line,pYards,yards_gained,Over_Picked,Over_Result,Win
21,J.Reynolds,DET,39.5,53.36,14.0,True,False,0
22,K.Raymond,DET,29.5,27.70,76.0,False,True,0
23,T.Hockenson,DET,42.5,35.19,80.0,False,True,0


In [79]:
w8_results['week']=8

### Week 10

In [80]:
week=10

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6



trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,A.Green,44.266282,0.000000,44.593091,23.525625,0.000000,0.000000,ARI,18.730833,16.0,0.0,20.0,7.0,0.0,0.0,7.166667,14.532343
1,E.Benjamin,5.466466,22.693298,18.000989,23.951324,40.473309,6.831867,ARI,19.569542,9.0,28.0,28.0,21.0,23.0,15.0,20.666667,19.963969
2,G.Dortch,6.667368,0.000000,0.000000,2.921464,19.001749,6.678697,ARI,5.878213,6.0,0.0,0.0,5.0,12.0,0.0,3.833333,9.297635
3,J.Conner,36.797706,30.075574,24.788528,16.184664,3.429786,29.326026,ARI,23.433714,29.0,26.0,18.0,22.0,8.0,19.0,20.333333,23.300494
4,M.Brown,54.376546,80.610047,124.714429,84.334507,92.688104,102.644805,ARI,89.894740,43.0,68.0,140.0,88.0,78.0,68.0,80.833333,75.225423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,T.McLaurin,43.355203,53.357518,23.122408,57.285837,61.407276,80.129581,WAS,53.109637,15.0,76.0,41.0,73.0,113.0,56.0,62.333333,60.079271
310,D.Milne,0.000000,0.000000,6.859129,3.224259,5.629935,2.721665,WAS,3.072498,0.0,0.0,5.0,4.0,4.0,6.0,3.166667,5.312514
311,J.Bates,0.000000,29.472004,23.062004,0.000000,5.433521,11.261831,WAS,11.538227,0.0,19.0,39.0,0.0,6.0,5.0,11.500000,11.120814
312,D.Brown,0.000000,0.000000,32.931990,49.563682,0.000000,0.000000,WAS,13.749279,0.0,0.0,6.0,105.0,0.0,0.0,18.500000,13.890367


In [81]:
w10_table = trailing_weeks.copy()


#w10_table = w10_table.reset_index().rename(columns={'index':'player'})

In [82]:
w10_table.set_index(['player','team'],inplace=True)

In [83]:
w10_odds = pd.read_csv('betting_lines/nfl-best-bets (5).csv').round(1)[['player','team','line']]






odds = w10_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w10_table = w10_table.merge(odds,right_index=True,left_index=True)
w10_table = w10_table.reset_index()

w10_table['abs. diff'] = abs(w10_table['pYards'] - w10_table['line'])

w10_table = w10_table.round(1)

w10_table['Over_Picked'] =w10_table['pYards']>w10_table ['line']

w10_table.sort_values('abs. diff',ascending=True).tail(25)

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
95,T.McLaurin,WAS,43.4,53.4,23.1,57.3,61.4,80.1,53.1,15.0,76.0,41.0,73.0,113.0,56.0,62.3,60.1,52.5,7.6,True
78,N.Gray,KC,14.0,0.0,13.8,19.6,7.2,33.3,14.6,26.0,0.0,7.0,13.0,10.0,45.0,16.8,20.2,12.5,7.7,True
75,N.Brown,DAL,48.2,40.3,51.7,11.9,13.1,60.9,37.7,91.0,54.0,61.0,5.0,10.0,50.0,45.2,39.4,31.5,7.9,True
42,G.Davis,BUF,56.6,24.2,73.0,64.1,71.2,49.1,56.4,37.0,13.0,171.0,74.0,35.0,33.0,60.5,53.7,45.5,8.2,True
24,D.Adams,LV,65.5,94.2,71.9,76.9,57.4,173.2,89.9,36.0,101.0,124.0,95.0,3.0,146.0,84.2,73.7,82.5,8.8,False
71,M.Gordon,DEN,32.6,0.0,16.8,0.0,21.9,22.0,15.6,29.0,0.0,49.0,0.0,17.0,23.0,19.7,20.4,11.5,8.9,True
16,C.Godwin,TB,77.9,47.2,119.9,95.9,68.0,68.8,79.6,55.0,61.0,95.0,43.0,75.0,36.0,60.8,55.6,64.5,8.9,False
31,D.Metcalf,SEA,91.7,102.9,59.0,53.4,81.9,30.6,69.9,64.0,149.0,88.0,34.0,55.0,37.0,71.2,69.8,60.5,9.3,True
36,D.Samuel,SF,32.5,64.9,41.3,69.5,80.7,36.7,54.3,44.0,73.0,115.0,20.0,79.0,42.0,62.2,59.8,50.5,9.3,True
14,B.Wright,DET,0.0,0.0,0.0,23.1,5.3,0.0,4.7,0.0,0.0,0.0,57.0,6.0,0.0,10.5,8.0,17.5,9.5,False


In [84]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==10]


100%|█████████████████████████████████████████████| 1/1 [00:07<00:00,  7.02s/it]


In [85]:
results

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
18175,55,2022_10_ARI_LA,2022111310,LA,ARI,REG,10,LA,home,ARI,...,0,1,0.524612,0.781011,6.208278,5.0,0.516835,0.241129,0.522348,47.765237
18176,79,2022_10_ARI_LA,2022111310,LA,ARI,REG,10,LA,home,ARI,...,0,1,0.292603,NaN,NaN,NaN,NaN,NaN,0.360472,-36.047161
18177,100,2022_10_ARI_LA,2022111310,LA,ARI,REG,10,LA,home,ARI,...,1,1,-0.520952,0.693845,7.029796,6.0,0.611832,0.231083,0.472686,52.731380
18178,129,2022_10_ARI_LA,2022111310,LA,ARI,REG,10,LA,home,ARI,...,0,1,-0.774751,NaN,NaN,NaN,NaN,NaN,0.551256,NaN
18179,152,2022_10_ARI_LA,2022111310,LA,ARI,REG,10,LA,home,ARI,...,0,1,-0.105223,NaN,NaN,NaN,NaN,NaN,0.764612,-76.461166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20063,3824,2022_10_WAS_PHI,2022111400,PHI,WAS,REG,10,WAS,away,PHI,...,0,0,-0.353920,NaN,NaN,NaN,NaN,NaN,0.061209,-6.120914
20064,3855,2022_10_WAS_PHI,2022111400,PHI,WAS,REG,10,WAS,away,PHI,...,0,0,-0.544183,NaN,NaN,NaN,NaN,NaN,0.176981,-17.698051
20065,3883,2022_10_WAS_PHI,2022111400,PHI,WAS,REG,10,WAS,away,PHI,...,0,0,0.958841,NaN,NaN,NaN,NaN,NaN,0.624016,37.598377
20066,3920,2022_10_WAS_PHI,2022111400,PHI,WAS,REG,10,WAS,away,PHI,...,0,0,-1.198526,NaN,NaN,NaN,NaN,NaN,0.031269,-3.126898


In [86]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w10_table = w10_table.merge(results)

w10_table['Over_Result'] = w10_table['yards_gained']>w10_table['line']

w10_table['Win'] = (w10_table['Over_Result'] == w10_table['Over_Picked'])*1


w10_table['week'] = 10


In [87]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

30.6534277296838
-----
Root Mean Square Error - Betting Line:

29.492148821428106


In [88]:
w10_table[w10_table['team']=='IND']

,player,team,1,2,3,4,5,6,xYards/game,1a,...,6a,yards/game,pYards,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
8,A.Pierce,IND,56.0,69.2,52.9,34.0,50.1,10.9,45.5,80.0,...,23.0,55.8,46.5,27.5,19.0,True,0.0,False,0,10
55,J.Taylor,IND,37.3,5.8,28.3,16.4,39.4,0.0,21.2,14.0,...,0.0,11.8,17.1,16.5,0.6,True,16.0,False,0,10
70,M.Pittman,IND,30.1,62.8,109.1,63.4,58.8,43.3,61.3,31.0,...,22.0,59.5,55.2,53.5,1.7,True,53.0,False,0,10
76,P.Campbell,IND,30.9,21.6,69.0,63.7,11.3,36.1,38.8,43.0,...,15.0,41.7,40.2,25.5,14.7,True,76.0,True,1,10


### Week 11

In [89]:
week=11

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6



trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,A.Green,0.000000,44.593091,23.525625,0.000000,0.000000,7.875284,ARI,12.665667,0.0,20.0,7.0,0.0,0.0,10.0,6.166667,12.141752
1,E.Benjamin,22.693298,18.000989,23.951324,40.473309,6.831867,0.000000,ARI,18.658465,28.0,28.0,21.0,23.0,15.0,0.0,19.166667,17.554839
2,G.Dortch,0.000000,0.000000,2.921464,19.001749,6.678697,0.000000,ARI,4.766985,0.0,0.0,5.0,12.0,0.0,0.0,2.833333,7.216025
3,J.Conner,30.075574,24.788528,16.184664,3.429786,29.326026,12.881582,ARI,19.447693,26.0,18.0,22.0,8.0,19.0,17.0,18.333333,20.397291
4,M.Brown,54.376546,80.610047,124.714429,84.334507,92.688104,102.644805,ARI,89.894740,43.0,68.0,140.0,88.0,78.0,68.0,80.833333,75.225423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,D.Milne,0.000000,6.859129,3.224259,5.629935,2.721665,0.000000,WAS,3.072498,0.0,5.0,4.0,4.0,6.0,0.0,3.166667,4.894250
332,J.Bates,29.472004,23.062004,0.000000,5.433521,11.261831,0.000000,WAS,11.538227,19.0,39.0,0.0,6.0,5.0,0.0,11.500000,11.648472
333,D.Brown,0.000000,32.931990,49.563682,0.000000,0.000000,9.255954,WAS,15.291938,0.0,6.0,105.0,0.0,0.0,15.0,21.000000,16.386785
334,C.Sims,21.073333,13.438437,5.174522,21.748848,7.340547,0.000000,WAS,11.462615,7.0,6.0,12.0,21.0,12.0,0.0,9.666667,10.666343


In [90]:
w11_table = trailing_weeks.copy()

#w11_table = w11_table.reset_index().rename(columns={'index':'player'})

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (8).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


def o_u(over):
  if over == True:
    return 'Over'
  elif over == False:
    return 'Under'



In [91]:
tnf = w11_table[(w11_table['team']=='GB')|(w11_table['team']=='TEN')]


In [92]:
tnf

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,A.Dillon,GB,0.0,36.4,0.0,5.0,27.4,0.0,11.5,0.0,11.0,0.0,9.0,10.0,0.0,5.0,10.9,8.5,2.4,True
1,A.Hooper,TEN,5.4,4.5,28.0,5.6,24.3,50.9,19.8,4.0,7.0,56.0,5.0,34.0,41.0,24.5,24.7,23.5,1.2,True
2,A.Jones,GB,15.1,22.2,59.6,23.0,10.5,10.3,23.4,17.0,25.0,53.0,14.0,20.0,18.0,24.5,24.9,24.5,0.4,True
4,A.Lazard,GB,85.5,67.3,90.2,48.8,85.0,25.5,67.1,116.0,35.0,76.0,55.0,87.0,45.0,69.0,60.0,49.5,10.5,True
9,C.Hollister,TEN,0.0,0.0,15.2,0.0,0.0,0.0,2.5,0.0,0.0,32.0,0.0,0.0,0.0,5.3,5.2,0.5,4.7,True
12,C.Okonkwo,TEN,21.1,0.0,0.0,16.9,18.3,17.4,12.3,38.0,0.0,0.0,3.0,48.0,41.0,21.7,18.8,10.5,8.3,True
16,C.Watson,GB,15.8,26.6,4.2,4.4,17.0,73.7,23.6,9.0,8.0,1.0,12.0,24.0,107.0,26.8,32.8,43.5,10.7,False
19,D.Henry,TEN,27.8,10.3,16.6,5.6,0.0,19.5,13.3,33.0,30.0,10.0,9.0,0.0,14.0,16.0,15.3,11.5,3.8,True
41,N.Westbrook-Ikhine,TEN,0.0,27.5,0.0,0.0,0.0,66.3,15.6,0.0,72.0,0.0,0.0,0.0,119.0,31.8,29.1,24.5,4.6,True
44,R.Cobb,GB,30.6,25.8,12.6,46.2,103.9,6.0,37.5,14.0,37.0,57.0,42.0,99.0,8.0,42.8,35.4,21.5,13.9,True


In [93]:
w11_table = trailing_weeks.copy()

#w11_table = w11_table.reset_index().rename(columns={'index':'player'})

w11_table.set_index(['player','team'],inplace=True)

w11_odds = pd.read_csv('betting_lines/nfl-best-bets (9).csv').round(1)[['player','team','line']]






odds = w11_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

w11_table = w11_table.merge(odds,right_index=True,left_index=True)
w11_table = w11_table.reset_index()

w11_table['abs. diff'] = abs(w11_table['pYards'] - w11_table['line'])

w11_table = w11_table.round(1)

w11_table['Over_Picked'] =w11_table['pYards']>w11_table ['line']


w11_table = w11_table.append(tnf)



In [94]:
snf = trailing_weeks.copy()


#snf = snf.reset_index().rename(columns={'index':'player'})

snf.set_index(['player','team'],inplace=True)

snf_odds = pd.read_csv('betting_lines/nfl-best-bets (10).csv').round(1)[['player','team','line']]






odds = snf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

snf = snf.merge(odds,right_index=True,left_index=True)
snf = snf.reset_index()

snf['abs. diff'] = abs(snf['pYards'] - snf['line'])

snf = snf.round(1)

snf['Over_Picked'] =snf['pYards']>snf ['line']


primetime = snf[(snf['team']=='KC')|(snf['team']=='LAC')]

In [95]:
w11_table = w11_table.append(primetime)

In [96]:
mnf = trailing_weeks.copy()


#mnf = mnf.reset_index().rename(columns={'index':'player'})

mnf.set_index(['player','team'],inplace=True)

mnf_odds = pd.read_csv('betting_lines/nfl-best-bets (11).csv').round(1)[['player','team','line']]






odds = mnf_odds.copy()

odds['team'] = odds['team'].apply(lambda x:team_changer(x))


odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])


odds['player'] = odds['player'].apply(lambda x:name_changer(x))


odds.set_index(['player','team'],inplace=True)

mnf = mnf.merge(odds,right_index=True,left_index=True)
mnf = mnf.reset_index()

mnf['abs. diff'] = abs(mnf['pYards'] - mnf['line'])

mnf = mnf.round(1)
mnf['Over_Picked'] =mnf['pYards']>mnf ['line']


mnf = mnf[(mnf['team']=='ARI')|(mnf['team']=='SF')]

In [97]:
mnf

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,B.Aiyuk,SF,24.0,40.6,89.4,74.5,45.3,64.9,56.4,37.0,58.0,83.0,82.0,81.0,84.0,70.8,62.3,54.5,7.8,True
1,D.Samuel,SF,64.9,41.3,69.5,80.7,36.7,45.4,56.4,73.0,115.0,20.0,79.0,42.0,24.0,58.8,52.6,52.5,0.1,True
2,G.Kittle,SF,25.8,42.2,63.9,70.0,30.1,20.5,42.1,24.0,47.0,83.0,98.0,39.0,21.0,52.0,44.0,41.5,2.5,True
3,J.Conner,ARI,30.1,24.8,16.2,3.4,29.3,12.9,19.4,26.0,18.0,22.0,8.0,19.0,17.0,18.3,20.4,19.5,0.9,True
4,J.Jennings,SF,19.7,33.2,13.4,18.7,27.0,29.5,23.6,5.0,22.0,45.0,18.0,24.0,40.0,25.7,27.8,14.5,13.3,True
5,K.Juszczyk,SF,13.2,5.8,9.6,26.7,25.3,0.0,13.4,24.0,35.0,27.0,33.0,34.0,0.0,25.5,16.4,7.5,8.9,True
6,R.Moore,ARI,49.3,70.8,15.4,46.4,71.0,87.0,56.6,68.0,49.0,31.0,92.0,69.0,94.0,67.2,58.4,59.5,1.1,False
7,T.McBride,ARI,0.0,0.0,0.0,0.0,0.0,5.6,0.9,0.0,0.0,0.0,0.0,0.0,7.0,1.2,5.0,19.5,14.5,False


In [98]:
w11_table = w11_table.append(mnf)

In [99]:
w11_table[(w11_table['team']=='ARI')|(w11_table['team']=='SF')]

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,B.Aiyuk,SF,24.0,40.6,89.4,74.5,45.3,64.9,56.4,37.0,58.0,83.0,82.0,81.0,84.0,70.8,62.3,54.5,7.8,True
1,D.Samuel,SF,64.9,41.3,69.5,80.7,36.7,45.4,56.4,73.0,115.0,20.0,79.0,42.0,24.0,58.8,52.6,52.5,0.1,True
2,G.Kittle,SF,25.8,42.2,63.9,70.0,30.1,20.5,42.1,24.0,47.0,83.0,98.0,39.0,21.0,52.0,44.0,41.5,2.5,True
3,J.Conner,ARI,30.1,24.8,16.2,3.4,29.3,12.9,19.4,26.0,18.0,22.0,8.0,19.0,17.0,18.3,20.4,19.5,0.9,True
4,J.Jennings,SF,19.7,33.2,13.4,18.7,27.0,29.5,23.6,5.0,22.0,45.0,18.0,24.0,40.0,25.7,27.8,14.5,13.3,True
5,K.Juszczyk,SF,13.2,5.8,9.6,26.7,25.3,0.0,13.4,24.0,35.0,27.0,33.0,34.0,0.0,25.5,16.4,7.5,8.9,True
6,R.Moore,ARI,49.3,70.8,15.4,46.4,71.0,87.0,56.6,68.0,49.0,31.0,92.0,69.0,94.0,67.2,58.4,59.5,1.1,False
7,T.McBride,ARI,0.0,0.0,0.0,0.0,0.0,5.6,0.9,0.0,0.0,0.0,0.0,0.0,7.0,1.2,5.0,19.5,14.5,False


In [100]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
31,J.Meyers,NE,103.7,72.6,39.5,15.7,81.3,43.7,59.4,95.0,111.0,60.0,34.0,60.0,42.0,67.0,71.1,48.5,22.6,True
34,K.Osborn,MIN,13.7,39.6,30.9,33.4,0.0,80.6,33.0,6.0,41.0,18.0,8.0,0.0,35.0,18.0,47.7,28.5,19.2,True
19,D.Peoples-Jones,CLE,69.9,60.5,42.5,41.7,40.3,92.0,57.8,71.0,50.0,74.0,71.0,81.0,99.0,74.3,65.0,46.5,18.5,True
51,T.Higgins,CIN,84.7,0.0,62.5,65.2,45.8,56.5,52.5,124.0,0.0,47.0,93.0,49.0,60.0,62.2,58.5,74.5,16.0,False
2,A.St. Brown,DET,77.0,62.4,32.0,75.1,79.4,76.6,67.1,116.0,73.0,18.0,69.0,55.0,119.0,75.0,65.5,80.5,15.0,False
7,T.McBride,ARI,0.0,0.0,0.0,0.0,0.0,5.6,0.9,0.0,0.0,0.0,0.0,0.0,7.0,1.2,5.0,19.5,14.5,False
44,R.Cobb,GB,30.6,25.8,12.6,46.2,103.9,6.0,37.5,14.0,37.0,57.0,42.0,99.0,8.0,42.8,35.4,21.5,13.9,True
4,J.Jennings,SF,19.7,33.2,13.4,18.7,27.0,29.5,23.6,5.0,22.0,45.0,18.0,24.0,40.0,25.7,27.8,14.5,13.3,True
37,M.Gordon,DEN,0.0,16.8,0.0,21.9,22.0,29.1,15.0,0.0,49.0,0.0,17.0,23.0,46.0,22.5,22.2,9.5,12.7,True
35,K.Pitts,ATL,46.1,16.6,26.1,52.5,83.8,79.5,50.8,25.0,19.0,9.0,80.0,27.0,28.0,31.3,49.1,36.5,12.6,True


In [101]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR
    data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==11]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w11_table = w11_table.merge(results)

w11_table['Over_Result'] = w11_table['yards_gained']>w11_table['line']

w11_table['Win'] = (w11_table['Over_Result'] == w11_table['Over_Picked'])*1


w11_table['week'] = 11

100%|█████████████████████████████████████████████| 1/1 [00:06<00:00,  6.82s/it]


In [102]:
w11_table[w11_table['abs. diff']>=10].sort_values('abs. diff',ascending=False)['Win'].mean()

0.5555555555555556

In [103]:

import math
MSE = np.square(np.subtract(w10_table['pYards'],w10_table['yards_gained'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:\n")
print(RMSE)

import math
MSE = np.square(np.subtract(w10_table['yards_gained'],w10_table['line'])).mean() 
 
print('-----')
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting Line:\n")
print(RMSE)

Root Mean Square Error - Projections:

30.6534277296838
-----
Root Mean Square Error - Betting Line:

29.492148821428106


### Week 12

In [104]:
week=12

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,A.Green,44.593091,23.525625,0.000000,0.000000,7.875284,62.935232,ARI,23.154872,20.0,7.0,0.0,0.0,10.0,50.0,14.500000,24.040473
1,E.Benjamin,22.693298,18.000989,23.951324,40.473309,6.831867,0.000000,ARI,18.658465,28.0,28.0,21.0,23.0,15.0,0.0,19.166667,17.554839
2,G.Dortch,0.000000,2.921464,19.001749,6.678697,0.000000,52.792229,ARI,13.565690,0.0,5.0,12.0,0.0,0.0,103.0,20.000000,24.558376
3,J.Conner,24.788528,16.184664,3.429786,29.326026,12.881582,25.628807,ARI,18.706565,18.0,22.0,8.0,19.0,17.0,12.0,16.000000,20.643187
4,M.Brown,54.376546,80.610047,124.714429,84.334507,92.688104,102.644805,ARI,89.894740,43.0,68.0,140.0,88.0,78.0,68.0,80.833333,75.225423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,D.Milne,6.859129,3.224259,5.629935,2.721665,0.000000,0.000000,WAS,3.072498,5.0,4.0,4.0,6.0,0.0,0.0,3.166667,4.894250
354,J.Bates,23.062004,0.000000,5.433521,11.261831,0.000000,0.000000,WAS,6.626226,39.0,0.0,6.0,5.0,0.0,0.0,8.333333,7.393733
355,D.Brown,32.931990,49.563682,0.000000,0.000000,9.255954,0.000000,WAS,15.291938,6.0,105.0,0.0,0.0,15.0,0.0,21.000000,15.799420
356,C.Sims,13.438437,5.174522,21.748848,7.340547,0.000000,11.751591,WAS,9.908991,6.0,12.0,21.0,12.0,0.0,17.0,11.333333,11.216241


In [105]:


w12_table = trailing_weeks.copy()

 
#w12_table = w12_table.reset_index().rename(columns={'index':'player'})
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (12).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 



In [106]:
tg = w12_table[w12_table['team']!='CAR'].copy()

In [107]:


w12_table = trailing_weeks.copy()

 
#w12_table = w12_table.reset_index().rename(columns={'index':'player'})
 
w12_table.set_index(['player','team'],inplace=True)
 
w12_odds = pd.read_csv('betting_lines/nfl-best-bets (13).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w12_table = w12_table.merge(odds,right_index=True,left_index=True)
w12_table = w12_table.reset_index()
 
w12_table['abs. diff'] = abs(w12_table['pYards'] - w12_table['line'])
 
w12_table = w12_table.round(1)
 
w12_table['Over_Picked'] =w12_table['pYards']>w12_table ['line']

w12_table = w12_table.append(tg)


In [108]:


mnf12 = trailing_weeks.copy()

 
#mnf12 = mnf12.reset_index().rename(columns={'index':'player'})
 
mnf12.set_index(['player','team'],inplace=True)
 
mnf12_odds = pd.read_csv('betting_lines/nfl-best-bets (14).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf12_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf12 = mnf12.merge(odds,right_index=True,left_index=True)
mnf12 = mnf12.reset_index()
 
mnf12['abs. diff'] = abs(mnf12['pYards'] - mnf12['line'])
 
mnf12 = mnf12.round(1)
 
mnf12['Over_Picked'] =mnf12['pYards']>mnf12 ['line']

w12_table = w12_table.append(mnf12)


In [109]:
mnf12

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,A.Pierce,IND,52.9,34.0,50.1,10.9,0.0,69.3,36.2,49.0,37.0,65.0,23.0,0.0,28.0,33.7,34.5,35.5,1.0,False
1,D.Johnson,PIT,86.2,39.3,69.0,47.7,44.2,28.9,52.6,60.0,28.0,42.0,35.0,63.0,21.0,41.5,46.8,45.5,1.3,True
2,G.Olszewski,PIT,0.0,0.0,0.0,0.0,5.1,18.1,3.9,0.0,0.0,0.0,0.0,4.0,39.0,7.2,10.5,12.5,2.0,False
3,G.Pickens,PIT,54.6,37.8,36.8,0.0,34.3,53.0,36.1,83.0,27.0,61.0,0.0,32.0,83.0,47.7,40.8,43.5,2.7,False
4,J.Taylor,IND,28.3,16.4,39.4,0.0,7.5,21.7,18.9,20.0,1.0,27.0,0.0,16.0,10.0,12.3,18.9,13.5,5.4,True
5,M.Alie-Cox,IND,0.0,0.0,0.0,13.0,0.0,5.6,3.1,0.0,0.0,0.0,13.0,0.0,14.0,4.5,6.4,13.5,7.1,False
6,M.Pittman,IND,109.1,63.4,58.8,43.3,65.2,51.1,65.2,134.0,58.0,53.0,22.0,53.0,75.0,65.8,61.7,67.5,5.8,False
7,N.Harris,PIT,19.9,13.7,19.2,26.5,0.0,35.9,19.2,16.0,7.0,15.0,26.0,0.0,26.0,15.0,20.7,17.5,3.2,True
8,P.Campbell,IND,69.0,63.7,11.3,36.1,56.1,45.7,47.0,57.0,70.0,43.0,15.0,76.0,67.0,54.7,49.3,48.5,0.8,True
9,P.Freiermuth,PIT,67.5,10.1,67.7,49.3,49.4,90.1,55.7,85.0,12.0,75.0,57.0,36.0,79.0,57.3,54.4,48.5,5.9,True


In [110]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==12]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w12_table = w12_table.merge(results)

w12_table['Over_Result'] = w12_table['yards_gained']>w12_table['line']

w12_table['Win'] = (w12_table['Over_Result'] == w12_table['Over_Picked'])*1


w12_table['week'] = 12

100%|█████████████████████████████████████████████| 1/1 [00:06<00:00,  6.61s/it]


### Week 13

In [111]:
week=13

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])



trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,A.Green,23.525625,0.000000,0.000000,7.875284,62.935232,21.355398,ARI,19.281923,7.0,0.0,0.0,10.0,50.0,5.0,12.000000,20.100815
1,E.Benjamin,22.693298,18.000989,23.951324,40.473309,6.831867,0.000000,ARI,18.658465,28.0,28.0,21.0,23.0,15.0,0.0,19.166667,17.554839
2,G.Dortch,0.000000,2.921464,19.001749,6.678697,0.000000,52.792229,ARI,13.565690,0.0,5.0,12.0,0.0,0.0,103.0,20.000000,24.558376
3,J.Conner,16.184664,3.429786,29.326026,12.881582,25.628807,12.399024,ARI,16.641648,22.0,8.0,19.0,17.0,12.0,20.0,16.333333,18.991245
4,M.Brown,80.610047,124.714429,84.334507,92.688104,102.644805,58.449978,ARI,90.573645,68.0,140.0,88.0,78.0,68.0,46.0,81.333333,73.159101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,D.Milne,6.859129,3.224259,5.629935,2.721665,0.000000,0.000000,WAS,3.072498,5.0,4.0,4.0,6.0,0.0,0.0,3.166667,4.894250
367,J.Bates,0.000000,5.433521,11.261831,0.000000,0.000000,22.468707,WAS,6.527343,0.0,6.0,5.0,0.0,0.0,24.0,5.833333,10.881197
368,D.Brown,49.563682,0.000000,0.000000,9.255954,0.000000,8.716499,WAS,11.256022,105.0,0.0,0.0,15.0,0.0,17.0,22.833333,10.951096
369,C.Sims,5.174522,21.748848,7.340547,0.000000,11.751591,0.000000,WAS,7.669251,12.0,21.0,12.0,0.0,17.0,0.0,10.333333,10.813016


In [112]:
tnf

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,A.Dillon,GB,0.0,36.4,0.0,5.0,27.4,0.0,11.5,0.0,11.0,0.0,9.0,10.0,0.0,5.0,10.9,8.5,2.4,True
1,A.Hooper,TEN,5.4,4.5,28.0,5.6,24.3,50.9,19.8,4.0,7.0,56.0,5.0,34.0,41.0,24.5,24.7,23.5,1.2,True
2,A.Jones,GB,15.1,22.2,59.6,23.0,10.5,10.3,23.4,17.0,25.0,53.0,14.0,20.0,18.0,24.5,24.9,24.5,0.4,True
4,A.Lazard,GB,85.5,67.3,90.2,48.8,85.0,25.5,67.1,116.0,35.0,76.0,55.0,87.0,45.0,69.0,60.0,49.5,10.5,True
9,C.Hollister,TEN,0.0,0.0,15.2,0.0,0.0,0.0,2.5,0.0,0.0,32.0,0.0,0.0,0.0,5.3,5.2,0.5,4.7,True
12,C.Okonkwo,TEN,21.1,0.0,0.0,16.9,18.3,17.4,12.3,38.0,0.0,0.0,3.0,48.0,41.0,21.7,18.8,10.5,8.3,True
16,C.Watson,GB,15.8,26.6,4.2,4.4,17.0,73.7,23.6,9.0,8.0,1.0,12.0,24.0,107.0,26.8,32.8,43.5,10.7,False
19,D.Henry,TEN,27.8,10.3,16.6,5.6,0.0,19.5,13.3,33.0,30.0,10.0,9.0,0.0,14.0,16.0,15.3,11.5,3.8,True
41,N.Westbrook-Ikhine,TEN,0.0,27.5,0.0,0.0,0.0,66.3,15.6,0.0,72.0,0.0,0.0,0.0,119.0,31.8,29.1,24.5,4.6,True
44,R.Cobb,GB,30.6,25.8,12.6,46.2,103.9,6.0,37.5,14.0,37.0,57.0,42.0,99.0,8.0,42.8,35.4,21.5,13.9,True


In [113]:
w13_table = trailing_weeks.copy()

 
#w13_table = w13_table.reset_index().rename(columns={'index':'player'})
 
w13_table.set_index(['player','team'],inplace=True)
 
w13_odds = pd.read_csv('betting_lines/nfl-best-bets (20).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w13_table = w13_table.merge(odds,right_index=True,left_index=True)
w13_table = w13_table.reset_index()
 
w13_table['abs. diff'] = abs(w13_table['pYards'] - w13_table['line'])
 
w13_table = w13_table.round(1)
 
w13_table['Over_Picked'] =w13_table['pYards']>w13_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 


#w13_table['mean_diff'] = abs(w13_table['xYards/game']-w13_table['line'])

tnf.append(w13_table[(w13_table['team']!='BUF')&(w13_table['team']!='NE')])

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,A.Dillon,GB,0.0,36.4,0.0,5.0,27.4,0.0,11.5,0.0,11.0,0.0,9.0,10.0,0.0,5.0,10.9,8.5,2.4,True
1,A.Hooper,TEN,5.4,4.5,28.0,5.6,24.3,50.9,19.8,4.0,7.0,56.0,5.0,34.0,41.0,24.5,24.7,23.5,1.2,True
2,A.Jones,GB,15.1,22.2,59.6,23.0,10.5,10.3,23.4,17.0,25.0,53.0,14.0,20.0,18.0,24.5,24.9,24.5,0.4,True
4,A.Lazard,GB,85.5,67.3,90.2,48.8,85.0,25.5,67.1,116.0,35.0,76.0,55.0,87.0,45.0,69.0,60.0,49.5,10.5,True
9,C.Hollister,TEN,0.0,0.0,15.2,0.0,0.0,0.0,2.5,0.0,0.0,32.0,0.0,0.0,0.0,5.3,5.2,0.5,4.7,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,T.McLaurin,WAS,57.3,61.4,80.1,88.6,65.1,49.5,67.0,73.0,113.0,56.0,128.0,55.0,48.0,78.8,68.4,63.5,4.9,True
116,T.Pollard,DAL,15.7,12.1,6.1,31.1,38.5,7.5,18.5,8.0,26.0,16.0,13.0,109.0,1.0,28.8,21.9,15.5,6.4,True
117,T.Sherfield,MIA,25.4,29.4,14.4,25.8,43.6,57.2,32.6,6.0,44.0,25.0,27.0,63.0,33.0,33.0,30.9,21.5,9.4,True
118,W.Dissly,SEA,5.8,28.5,8.4,20.7,14.3,29.0,17.8,12.0,45.0,12.0,24.0,28.0,17.0,23.0,21.9,18.5,3.4,True


In [114]:
w13_table = w13_table[w13_table['team']!='IND']

In [115]:
w13_table = w13_table[w13_table['team']!='DAL']

w13_table = w13_table[w13_table['team']!='NO']

w13_table = w13_table[w13_table['team']!='TB']

In [116]:
w13 = trailing_weeks.sort_values('pYards',ascending=False)


#w13[(w13['team']=='IND')|(w13['team']=='DAL')]

In [117]:
snf13 = trailing_weeks.copy()


 
#snf13 = snf13.reset_index().rename(columns={'index':'player'})
 
snf13.set_index(['player','team'],inplace=True)
 
snf13_odds = pd.read_csv('betting_lines/nfl-best-bets (21).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = snf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
snf13 = snf13.merge(odds,right_index=True,left_index=True)
snf13 = snf13.reset_index()
 
snf13['abs. diff'] = abs(snf13['pYards'] - snf13['line'])
 
snf13 = snf13.round(1)
 
snf13['Over_Picked'] =snf13['pYards']>snf13 ['line']

In [118]:
mnf13 = trailing_weeks.copy()

 
#mnf13 = mnf13.reset_index().rename(columns={'index':'player'})
 
mnf13.set_index(['player','team'],inplace=True)
 
mnf13_odds = pd.read_csv('betting_lines/nfl-best-bets (23).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf13_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf13 = mnf13.merge(odds,right_index=True,left_index=True)
mnf13 = mnf13.reset_index()
 
mnf13['abs. diff'] = abs(mnf13['pYards'] - mnf13['line'])
 
mnf13 = mnf13.round(1)
 
mnf13['Over_Picked'] = mnf13['pYards']>mnf13 ['line']

In [119]:
#mnf13

In [120]:
w13_table = w13_table.append(snf13)

w13_table = w13_table.append(mnf13)

In [121]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'

    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]
    results = data[data['week']==13]


results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained']]

w13_table = w13_table.merge(results)

w13_table['Over_Result'] = w13_table['yards_gained']>w13_table['line']

w13_table['Win'] = (w13_table['Over_Result'] == w13_table['Over_Picked'])*1


w13_table['week'] = 13

100%|█████████████████████████████████████████████| 1/1 [00:06<00:00,  7.00s/it]


In [122]:
w13_table[w13_table['abs. diff']>=10].sort_values('abs. diff',ascending=False).head(50)

,player,team,1,2,3,4,5,6,xYards/game,1a,...,6a,yards/game,pYards,line,abs. diff,Over_Picked,yards_gained,Over_Result,Win,week
22,D.Adams,LV,76.9,57.4,173.2,101.8,106.1,88.2,100.6,95.0,...,74.0,97.5,64.6,95.5,30.9,False,177.0,True,0,13
0,A.Brown,PHI,54.5,109.8,50.4,41.3,47.2,43.3,57.7,67.0,...,46.0,65.8,56.2,74.5,18.3,False,119.0,True,0,13
45,G.Wilson,NYJ,33.3,23.3,52.2,63.6,20.2,53.4,41.0,8.0,...,95.0,57.7,45.8,62.5,16.7,False,162.0,True,0,13
117,J.Landry,NO,31.3,45.2,10.8,53.3,23.7,20.1,30.7,25.0,...,20.0,24.0,26.2,40.5,14.3,False,14.0,False,1,13
17,C.Okonkwo,TEN,0.0,16.9,18.3,17.4,17.7,31.1,16.9,0.0,...,35.0,26.3,24.6,11.5,13.1,True,68.0,True,1,13
100,Z.Jones,JAX,40.6,76.8,25.4,32.2,70.1,103.1,58.0,53.0,...,145.0,64.7,62.3,49.5,12.8,True,16.0,False,0,13
23,D.Carter,LAC,32.1,60.9,49.0,24.6,23.8,82.1,45.5,17.0,...,73.0,46.2,51.6,39.5,12.1,True,14.0,False,0,13
46,H.Bryant,CLE,29.5,16.4,0.0,12.5,40.3,10.0,18.1,18.0,...,9.0,16.3,18.5,30.5,12.0,False,6.0,False,1,13
19,C.Sutton,DEN,19.5,66.0,32.2,81.9,60.7,47.1,51.2,14.0,...,75.0,45.2,48.9,60.5,11.6,False,0.0,False,1,13
59,J.Waddle,MIA,101.9,53.2,74.9,71.3,53.6,118.0,78.8,129.0,...,85.0,93.2,77.1,65.5,11.6,True,9.0,False,0,13


### Week 14

In [123]:
week=14

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

trailing_weeks

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
0,A.Green,23.525625,0.000000,0.000000,7.875284,62.935232,21.355398,ARI,19.281923,7.0,0.0,0.0,10.0,50.0,5.0,12.000000,20.100815
1,E.Benjamin,22.693298,18.000989,23.951324,40.473309,6.831867,0.000000,ARI,18.658465,28.0,28.0,21.0,23.0,15.0,0.0,19.166667,17.554839
2,G.Dortch,0.000000,2.921464,19.001749,6.678697,0.000000,52.792229,ARI,13.565690,0.0,5.0,12.0,0.0,0.0,103.0,20.000000,24.558376
3,J.Conner,16.184664,3.429786,29.326026,12.881582,25.628807,12.399024,ARI,16.641648,22.0,8.0,19.0,17.0,12.0,20.0,16.333333,18.991245
4,M.Brown,80.610047,124.714429,84.334507,92.688104,102.644805,58.449978,ARI,90.573645,68.0,140.0,88.0,78.0,68.0,46.0,81.333333,73.159101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,J.Bates,5.433521,11.261831,0.000000,0.000000,22.468707,0.000000,WAS,6.527343,6.0,5.0,0.0,0.0,24.0,0.0,5.833333,8.541324
375,D.Brown,0.000000,0.000000,9.255954,0.000000,8.716499,0.000000,WAS,2.995409,0.0,0.0,15.0,0.0,17.0,0.0,5.333333,6.335641
376,C.Sims,21.748848,7.340547,0.000000,11.751591,0.000000,13.351835,WAS,9.032137,21.0,12.0,0.0,17.0,0.0,-2.0,8.000000,10.751582
377,C.Turner,0.000000,17.861484,0.000000,0.000000,0.000000,0.000000,WAS,2.976914,0.0,23.0,0.0,0.0,0.0,0.0,3.833333,5.212065


In [124]:


w14_table = trailing_weeks.copy()

 
#w14_table = w14_table.reset_index().rename(columns={'index':'player'})
 
w14_table.set_index(['player','team'],inplace=True)
 
w14_odds = pd.read_csv('betting_lines/nfl-best-bets (24).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w14_table = w14_table.merge(odds,on=['player','team'])
w14_table = w14_table.reset_index()
 
w14_table['abs. diff'] = abs(w14_table['pYards'] - w14_table['line'])
 
w14_table = w14_table.round(1)
 
w14_table['Over_Picked'] =w14_table['pYards']>w14_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
tnf14 = w14_table.copy()



In [125]:


w14_table = trailing_weeks.copy()

w14_table.set_index(['player','team'],inplace=True)
 
#w14_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
#w14_table.set_index(['player','team'],inplace=True)
 
w14_odds = pd.read_csv('betting_lines/nfl-best-bets (27).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
w14_table = w14_table.merge(odds,on=['player','team'])
w14_table = w14_table.reset_index()
 
w14_table['abs. diff'] = abs(w14_table['pYards'] - w14_table['line'])
 
w14_table = w14_table.round(1)
 
w14_table['Over_Picked'] =w14_table['pYards']>w14_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
w14_table = tnf14.append(w14_table)

w14_table = w14_table[(w14_table['team']!='LAC') & (w14_table['team']!='MIA')]

w14_table = w14_table[(w14_table['team']!='ARI') & (w14_table['team']!='NE')]

In [126]:
w14_table

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,T.Higbee,LA,33.5,0.0,43.6,55.2,0.0,21.9,25.7,15.0,0.0,73.0,45.0,0.0,14.0,24.5,24.0,35.0,11.0,False
1,V.Jefferson,LA,0.0,0.0,21.1,43.8,38.0,47.2,25.0,0.0,0.0,27.0,41.0,29.0,39.0,22.7,26.8,37.5,10.7,False
2,D.Adams,LV,57.4,173.2,101.8,106.1,88.2,113.3,106.7,3.0,146.0,126.0,141.0,74.0,177.0,111.2,90.7,89.5,1.2,True
3,J.Jacobs,LV,25.3,17.9,46.9,25.1,39.9,12.1,27.9,11.0,20.0,28.0,51.0,74.0,6.0,31.7,24.3,20.5,3.8,True
4,M.Hollins,LV,61.5,46.2,66.4,71.1,39.7,61.6,57.8,64.0,17.0,18.0,52.0,63.0,35.0,41.5,48.1,45.5,2.6,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,A.Hooper,TEN,5.6,24.3,50.9,18.8,23.8,38.6,27.0,5.0,34.0,41.0,36.0,30.0,22.0,28.0,27.1,25.5,1.6,True
83,C.Okonkwo,TEN,16.9,18.3,17.4,17.7,31.1,30.0,21.9,3.0,48.0,41.0,31.0,35.0,68.0,37.7,28.1,25.5,2.6,True
84,N.Westbrook-Ikhine,TEN,0.0,0.0,66.3,13.6,44.0,13.6,22.9,0.0,0.0,119.0,28.0,58.0,4.0,34.8,28.4,31.5,3.1,False
85,R.Woods,TEN,10.5,0.0,49.9,42.7,40.7,36.9,30.1,26.0,0.0,10.0,69.0,16.0,6.0,21.2,28.8,39.5,10.7,False


In [127]:


snf14 = trailing_weeks.copy()

 
 
snf14.set_index(['player','team'],inplace=True)
 
snf14_odds = pd.read_csv('betting_lines/nfl-best-bets (28).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = snf14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
snf14 = snf14.merge(odds,right_index=True,left_index=True)
snf14 = snf14.reset_index()
 
snf14['abs. diff'] = abs(snf14['pYards'] - snf14['line'])
 
snf14 = snf14.round(1)
 
snf14['Over_Picked'] =snf14['pYards']>snf14 ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
snf14 = snf14[(snf14['team']=='LAC')|(snf14['team']=='MIA')]

In [128]:


mnf14 = trailing_weeks.copy()

 
 
mnf14.set_index(['player','team'],inplace=True)
 
mnf14_odds = pd.read_csv('betting_lines/nfl-best-bets (30).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = mnf14_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
mnf14 = mnf14.merge(odds,right_index=True,left_index=True)
mnf14 = mnf14.reset_index()
 
mnf14['abs. diff'] = abs(mnf14['pYards'] - mnf14['line'])
 
mnf14 = mnf14.round(1)
 
mnf14['Over_Picked'] =mnf14['pYards']>mnf14 ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])


mnf14 = mnf14[(mnf14['team']=='NE')|(mnf14['team']=='ARI')]



In [129]:
mnf14

,player,team,1,2,3,4,5,6,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards,line,abs. diff,Over_Picked
0,D.Hopkins,ARI,114.8,108.3,37.0,96.2,76.7,72.2,84.2,103.0,159.0,36.0,98.0,91.0,87.0,95.7,77.3,76.5,0.8,True
1,D.Parker,NE,49.5,49.9,0.0,14.0,38.2,29.6,30.2,64.0,68.0,0.0,19.0,80.0,16.0,41.2,33.0,38.5,5.5,False
2,H.Henry,NE,17.5,11.5,32.9,6.1,39.5,31.8,23.2,12.0,22.0,50.0,20.0,63.0,13.0,30.0,26.7,29.5,2.8,False
3,J.Conner,ARI,16.2,3.4,29.3,12.9,25.6,12.4,16.6,22.0,8.0,19.0,17.0,12.0,20.0,16.3,19.0,18.5,0.5,True
4,J.Smith,NE,23.8,26.6,16.9,24.7,0.0,8.3,16.7,4.0,10.0,21.0,40.0,0.0,6.0,13.5,16.0,13.5,2.5,True
5,M.Brown,ARI,80.6,124.7,84.3,92.7,102.6,58.4,90.6,68.0,140.0,88.0,78.0,68.0,46.0,81.3,73.2,56.5,16.7,True
6,N.Agholor,NE,0.0,5.4,0.0,25.3,55.7,21.6,18.0,0.0,2.0,0.0,18.0,65.0,17.0,17.0,21.0,33.5,12.5,False
7,R.Stevenson,NE,46.0,44.9,37.7,34.1,53.1,39.7,42.6,59.0,72.0,10.0,56.0,76.0,24.0,49.5,42.0,36.5,5.5,True
8,T.Thornton,NE,56.3,14.9,38.2,11.7,0.0,32.5,25.6,19.0,13.0,5.0,5.0,0.0,31.0,12.2,22.5,22.5,0.0,False


In [130]:
w14_table = w14_table.append(snf14).append(mnf)

In [131]:
for YEAR in tqdm(range(2022,2023)):
    data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data = data.loc[data.season_type=='REG']
    data['season'] = YEAR

    data = data[data['posteam_type'].isna()==False]

    data.loc[data['pass']==1, 'play_type'] = 'pass'
    data.loc[data.rush==1, 'play_type'] = 'run'
    
    
    

    data.reset_index(drop=True, inplace=True)


    data = data[data['two_point_attempt']==0]

    data['home_implied_total'] = abs(data['total_line']/2 + data['spread_line']/2)
    data['away_implied_total'] = abs(data['total_line']/2 - data['spread_line']/2)
    implied_list = []
    for num in data.index:
        home_number = data.loc[num]['home_implied_total']
        away_number = data.loc[num]['away_implied_total']
        has_ball = data.loc[num]['posteam_type']
        number = total_finder(has_ball,home_number,away_number)
        implied_list.append(number)

    data['implied_posteam_total'] = implied_list
    
    
    
    data = data[data['two_point_attempt']==0]
    
    results = data[data['week']==14]
    results['middle'] = (results['pass_location']=='middle')*1 
    results['goal_to_go'] = (results['yardline_100']<=10)*1
    results = results[['receiver_player_name','posteam','yards_gained','down','yardline_100','ydstogo','implied_posteam_total','air_yards','middle','qb_hit']].set_index(['receiver_player_name','posteam'])
    results = results.dropna()
    
    results['xYards']=play_by_play_model.predict(results.drop(columns={'yards_gained'}))

    
    
    




100%|█████████████████████████████████████████████| 1/1 [00:47<00:00, 47.68s/it]


In [132]:
results = results.groupby(['receiver_player_name','posteam']).agg({'yards_gained':'sum','xYards':'sum'}).reset_index().rename(columns={'receiver_player_name':'player','posteam':'team'})[['player','team','yards_gained','xYards']]

w14_table = w14_table.merge(results)

w14_table['Over_Result'] = w14_table['yards_gained']>w14_table['line']

w14_table['Win'] = (w14_table['Over_Result'] == w14_table['Over_Picked'])*1


w14_table['week'] = 14

### Week 15

In [133]:
week=15

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])



In [134]:
beta_table = trailing_weeks[['player','team','pYards']]

beta_table.rename(columns={'pYards':'beta_yards'},inplace=True)

beta_table.to_csv('wk15_beta.csv')

In [135]:


weekly_table = trailing_weeks.copy()

weekly_table.set_index(['player','team'],inplace=True)
 
#w14_table.rename(columns={'receiver_player_name':'player'},inplace=True)
 
#w14_table.set_index(['player','team'],inplace=True)
 
w15_odds = pd.read_csv('betting_lines/nfl-best-bets (32).csv').round(1)[['player','team','line']]
 
 
 
 
 
 
odds = w15_odds.copy()
 
odds['team'] = odds['team'].apply(lambda x:team_changer(x))
 
 
odds['player'] = odds['player'].apply(lambda x: x.split(".")[0])+"."+odds['player'].apply(lambda x: x.split(".")[1][1:])
 
 
odds['player'] = odds['player'].apply(lambda x:name_changer(x))
 
 
odds.set_index(['player','team'],inplace=True)
 
weekly_table = weekly_table.merge(odds,on=['player','team'])
weekly_table = weekly_table.reset_index()
 
weekly_table['abs. diff'] = abs(weekly_table['pYards'] - weekly_table['line'])
 
weekly_table = weekly_table.round(1)
 
weekly_table['Over_Picked'] =weekly_table['pYards']>weekly_table ['line']
 
#w12_table['abs. median diff'] = abs(w12_table['xYards_median'] - w12_table['line'])

 
#weekly_table = tnf14.append(w14_table)

#w14_table = w14_table[(w14_table['team']!='LAC') & (w14_table['team']!='MIA')]

#w14_table = w14_table[(w14_table['team']!='ARI') & (w14_table['team']!='NE')]

In [136]:
w15_table = weekly_table.copy()

#w15_table.sort_values('abs. diff',ascending=False).head(25)

In [137]:
beta_table

,player,team,beta_yards
0,A.Green,ARI,19.327892
1,E.Benjamin,ARI,17.554839
2,G.Dortch,ARI,24.451754
3,J.Conner,ARI,21.793847
4,M.Brown,ARI,74.365549
...,...,...,...
381,J.Bates,WAS,8.541324
382,D.Brown,WAS,6.335641
383,C.Sims,WAS,10.751582
384,C.Turner,WAS,5.212065


### Week 16

In [138]:
week=16

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

beta_table = trailing_weeks[['player','team','pYards']]

beta_table.rename(columns={'pYards':'beta_yards'},inplace=True)

beta_table.to_csv('wk16_beta.csv')


In [139]:
trailing_weeks[trailing_weeks['team']=='JAX']

,player,1,2,3,4,5,6,team,xYards/game,1a,2a,3a,4a,5a,6a,yards/game,pYards
187,C.Kirk,55.981030,83.495952,64.256886,64.128883,43.643290,100.395967,JAX,68.650335,76.0,105.0,46.0,104.0,45.0,92.0,78.000000,71.563382
188,E.Engram,11.104855,28.682297,5.852599,44.114719,95.291796,66.268578,JAX,41.885807,8.0,14.0,4.0,30.0,162.0,62.0,46.666667,43.841496
189,J.Agnew,0.000000,24.534694,28.624843,0.000000,24.011625,17.321191,JAX,15.748726,0.0,4.0,37.0,0.0,36.0,12.0,14.833333,17.845562
190,J.Robinson,5.138070,0.000000,0.000000,7.373573,0.000000,0.000000,JAX,2.085274,1.0,0.0,0.0,5.0,0.0,0.0,1.000000,4.836707
191,M.Jones,48.793809,28.040991,16.138295,49.434565,9.678780,56.154860,JAX,34.706883,48.0,33.0,22.0,17.0,22.0,17.0,26.500000,31.341761
192,T.Etienne,11.044100,15.090533,0.000000,18.011984,0.000000,16.370074,JAX,10.086115,17.0,28.0,0.0,12.0,0.0,24.0,13.500000,13.873353
193,Z.Jones,32.168056,70.053765,103.082081,46.461400,83.840877,59.523302,JAX,65.854914,40.0,68.0,145.0,16.0,77.0,109.0,75.833333,64.614987
194,D.Arnold,7.341358,7.043860,0.000000,0.000000,16.191856,0.000000,JAX,5.096179,19.0,8.0,0.0,0.0,5.0,0.0,5.333333,8.541324
195,C.Manhertz,14.384371,0.000000,0.000000,0.000000,5.578261,0.000000,JAX,3.327105,7.0,0.0,0.0,0.0,21.0,0.0,4.666667,5.484775
196,J.Hasty,5.651851,9.911523,28.452473,0.000000,0.000000,3.385571,JAX,7.900236,5.0,-1.0,67.0,0.0,0.0,2.0,12.166667,9.244467


### Week 17

In [140]:
week=17

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

beta_table = trailing_weeks[['player','team','pYards']]

beta_table.rename(columns={'pYards':'beta_yards'},inplace=True)

beta_table.to_csv('wk17_beta.csv')


### Week 18

In [141]:
week=18

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

beta_table = trailing_weeks[['player','team','pYards']]

beta_table.rename(columns={'pYards':'beta_yards'},inplace=True)

beta_table.to_csv('wk18_beta.csv')


In [142]:
week=19

weekly_player_list = []
weekly_yard_list = []

for player in new_df['full_name'].unique():
    df_df = new_df[new_df['full_name']==player]
    yds = df_df.copy()
    name = list(df_df['receiver_player_name'])[0]
    team = list(df_df['posteam'])[0]

    df_df = df_df[df_df['week']<week].tail(6)[['xYards']]
    df_df.rename(columns={'xYards':name},inplace=True)
    df_df = df_df.transpose()
    if len(list(df_df.columns))==6:
        df_df.columns = [1,2,3,4,5,6]
        df_df['team']=team
        weekly_player_list.append(df_df)
    
    yd_df = yds[yds['week']<week].tail(6)[['yards_gained']] 
    yd_df.rename(columns={'yards_gained':name},inplace=True)
    yd_df = yd_df.transpose()
    if len(list(yd_df.columns))==6:
        
        yd_df.columns = ['1a','2a','3a','4a','5a','6a']
        yd_df['team']=team

        weekly_yard_list.append(yd_df)



trailing_weeks = pd.concat(weekly_player_list)

yardage_trailing = pd.concat(weekly_yard_list)


trailing_weeks =  trailing_weeks.reset_index().rename(columns={'index':'player'})

yardage_trailing = yardage_trailing.reset_index().rename(columns={'index':'player'})

trailing_weeks['xYards/game']=(trailing_weeks[1]+trailing_weeks[2]+trailing_weeks[3]+trailing_weeks[4]+trailing_weeks[5]+trailing_weeks[6])/6

trailing_weeks = trailing_weeks.merge(yardage_trailing, on=['player','team'])


trailing_weeks['yards/game']=(trailing_weeks['1a']+trailing_weeks['2a']+trailing_weeks['3a']+trailing_weeks['4a']+trailing_weeks['5a']+trailing_weeks['6a'])/6


trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6,'1a','2a','3a','4a','5a','6a','xYards/game','yards/game']])
#trailing_weeks['pYards'] = in_season_model.predict(trailing_weeks[[1,2,3,4,5,6]])

beta_table = trailing_weeks[['player','team','pYards']]

beta_table.rename(columns={'pYards':'beta_yards'},inplace=True)

beta_table.to_csv('WC_beta.csv')


In [154]:
beta_table

,player,team,beta_yards
0,A.Green,ARI,15.758783
1,E.Benjamin,ARI,14.338279
2,G.Dortch,ARI,35.346982
3,J.Conner,ARI,26.141655
4,M.Brown,ARI,53.640245
...,...,...,...
402,D.Brown,WAS,4.672596
403,C.Sims,WAS,8.257868
404,C.Turner,WAS,4.548942
405,B.Robinson,WAS,7.944407


## Overall Results

In [143]:
szn_results = w8_results.append(w9_table).append(w10_table).append(w11_table).append(w12_table).append(w13_table).append(w14_table)

szn_results['abs_diff'] = abs(szn_results['pYards']-szn_results['line'])

szn_results['smash'] = szn_results['abs_diff']>=10

smash_df = szn_results[szn_results['smash']==True]


In [144]:
print('R Values:\n')


print('Yards and projection: {}'.format(round(szn_results['pYards'].corr(szn_results['yards_gained']),3)))
print('Yards and betting line: {}'.format(round(szn_results['line'].corr(szn_results['yards_gained']),3)))
print('Projection and betting line: {}'.format(round(szn_results['pYards'].corr(szn_results['line']),3)))


R Values:

Yards and projection: 0.541
Yards and betting line: 0.6
Projection and betting line: 0.936


In [145]:


print('Overall Record: {}-{} ({}%)'.format(szn_results['Win'].sum(),szn_results.shape[0]-szn_results['Win'].sum(),round(szn_results['Win'].sum()/szn_results.shape[0]*100,1)))

print('Plays Record (10+ yard difference): {}-{} ({}%)'.format(smash_df['Win'].sum(),smash_df.shape[0]-smash_df['Win'].sum(),round(smash_df['Win'].sum()/smash_df.shape[0]*100,1)))

print('\n-----\n')
 
mae = mean_absolute_error(szn_results['pYards'], szn_results['yards_gained'])
    
print("Mean Absolute Error - Projections:")
print('{}\n'.format(round(mae,1)))


mae = mean_absolute_error(szn_results['yards_gained'],szn_results['line'])
 
print("Mean Absolute Error - Betting line:")
print(round(mae,1))

Overall Record: 358-324 (52.5%)
Plays Record (10+ yard difference): 51-68 (42.9%)

-----

Mean Absolute Error - Projections:
22.0

Mean Absolute Error - Betting line:
21.1


In [146]:
MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['pYards'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Projections:")
print('{}\n'.format(round(RMSE,1)))


MSE = np.square(np.subtract(szn_results['yards_gained'],szn_results['line'])).mean() 
 
RMSE = math.sqrt(MSE)
print("Root Mean Square Error - Betting line:")
print(round(RMSE,1))

Root Mean Square Error - Projections:
29.8

Root Mean Square Error - Betting line:
28.5


In [147]:
print('WEEKLY RESULTS')

print('\nOverall')
for week in szn_results['week'].unique():
    w = szn_results[szn_results['week']==week]['Win'].sum()
    p = szn_results[szn_results['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))


print('\nPlays')

for week in smash_df['week'].unique():
    w = smash_df[smash_df['week']==week]['Win'].sum()
    p = smash_df[smash_df['week']==week]['Win'].shape[0]
    l = p - w
    perc = round(w/p*100,1)
    print('Week {}: {}-{} ({}%)'.format(week,w,l,perc))

WEEKLY RESULTS

Overall
Week 8: 52-46 (53.1%)
Week 9: 42-45 (48.3%)
Week 10: 51-44 (53.7%)
Week 11: 44-34 (56.4%)
Week 12: 57-49 (53.8%)
Week 13: 62-60 (50.8%)
Week 14: 50-46 (52.1%)

Plays
Week 8: 6-7 (46.2%)
Week 9: 8-7 (53.3%)
Week 10: 3-11 (21.4%)
Week 11: 10-8 (55.6%)
Week 12: 12-13 (48.0%)
Week 13: 5-11 (31.2%)
Week 14: 7-11 (38.9%)


In [148]:
szn_results['Over_Picked'].value_counts()

True     433
False    249
Name: Over_Picked, dtype: int64

In [149]:
szn_results['Over_Result'].value_counts()

True     349
False    333
Name: Over_Result, dtype: int64

In [150]:
over_w = szn_results[szn_results['Over_Picked']==1]['Win'].mean()
under_w = szn_results[szn_results['Over_Picked']==0]['Win'].mean()

print('Overs: {}%'.format(round(over_w*100,1)))
print('Under: {}%'.format(round(under_w*100,1)))

Overs: 52.9%
Under: 51.8%


In [151]:
over_w = smash_df[smash_df['Over_Picked']==1]['Win'].mean()
under_w = smash_df[smash_df['Over_Picked']==0]['Win'].mean()

print('Smash Overs: {}%'.format(round(over_w*100,1)))
print('Smash Under: {}%'.format(round(under_w*100,1)))

Smash Overs: 40.6%
Smash Under: 46.0%


In [152]:
szn_results[(szn_results['week']==14)&(szn_results['smash']==True)]

,player,team,line,pYards,yards_gained,Over_Picked,Over_Result,Win,week,1,...,2a,3a,4a,5a,6a,yards/game,abs. diff,xYards,abs_diff,smash
0,T.Higbee,LA,35.0,24.0,11.0,False,False,1,14,33.5,...,0.0,73.0,45.0,0.0,14.0,24.5,11.0,15.914439,11.0,True
1,V.Jefferson,LA,37.5,26.8,44.0,False,True,0,14,0.0,...,0.0,27.0,41.0,29.0,39.0,22.7,10.7,38.453718,10.7,True
5,D.Robinson,BAL,30.5,41.6,52.0,True,True,1,14,0.0,...,64.0,12.0,128.0,17.0,41.0,43.7,11.1,41.138882,11.1,True
12,J.Cook,BUF,12.5,24.1,9.0,True,False,0,14,12.6,...,18.0,0.0,0.0,14.0,41.0,19.0,11.6,6.541074,11.6,True
17,T.Boyd,CIN,41.5,55.0,0.0,True,False,0,14,83.3,...,38.0,44.0,42.0,16.0,60.0,59.2,13.5,4.411993,13.5,True
28,K.Hinton,DEN,37.5,21.6,38.0,False,True,0,14,7.1,...,0.0,62.0,57.0,35.0,11.0,29.8,15.9,30.727275,15.9,True
43,J.Jefferson,MIN,94.5,76.8,233.0,False,True,0,14,70.1,...,115.0,193.0,33.0,139.0,45.0,103.8,17.7,135.455708,17.7,True
47,D.Slayton,NYG,45.5,67.2,42.0,True,False,0,14,55.2,...,66.0,95.0,86.0,63.0,90.0,76.3,21.7,23.593990,21.7,True
66,B.Aiyuk,SF,43.5,54.8,57.0,True,True,1,14,74.5,...,81.0,84.0,20.0,65.0,46.0,63.0,11.3,25.299250,11.3,True
75,C.Godwin,TB,59.5,72.7,54.0,True,False,0,14,95.9,...,75.0,36.0,71.0,110.0,63.0,66.3,13.2,60.436240,13.2,True


In [153]:
#szn_results['the_blend'] = (szn_results['xYards_median']+szn_results['pYards'])/2

#szn_results[['pYards','yards_gained','xYards/game','line','the_blend','xYards_median','gb_projection','linear_projection']].corr().sort_values('yards_gained',ascending=False)